In [4]:
%pip install playwright pandas openpyxl python-dateutil requests

import sys, subprocess; subprocess.run([sys.executable, "-m", "playwright", "install", "chromium"], check=True)

Note: you may need to restart the kernel to use updated packages.


CompletedProcess(args=['C:\\Users\\otavi\\anaconda3\\python.exe', '-m', 'playwright', 'install', 'chromium'], returncode=0)

In [19]:
# %% [markdown]
# Angra Infra → Vehicle → Financials → métricas e séries (IRR/DPI/RVPI/TVPI) + PME(CDI)
# Salva em: C:\Users\otaviobezerra\OneDrive - FUNCEF\Documentos\PBI_COREM\FIPS\angra_infra_e_ip.xlsx

# %%
# Rode uma vez se precisar instalar:
# %pip install playwright pandas openpyxl python-dateutil requests
# import sys, subprocess; subprocess.run([sys.executable, "-m", "playwright", "install", "chromium"], check=True)

# %%
import os, re, math, json, asyncio, datetime as dt
from pathlib import Path
import pandas as pd
import requests
from dateutil import parser as dtparse
from playwright.async_api import async_playwright

FUND_URL    = "https://pebay.info/backoffice/fund/angra-infra"
OUTPUT_DIR  = r"C:\Users\otavi\OneDrive\Documentos\Estudos_2025\COINP\PROJECOES\BANCODEDADOS\FIP"
OUTPUT_XLSX = os.path.join(OUTPUT_DIR, "angra_infra_e_ip.xlsx")
USER_DATA   = os.path.join(Path.home(), ".pw-angra")  # contexto persistente (login fica salvo)
HEADLESS    = False

os.makedirs(OUTPUT_DIR, exist_ok=True)
NOW = dt.datetime.now().strftime("%Y-%m-%d %H:%M:%S")

# ---------------- helpers ----------------
def clean(s):
    return re.sub(r"\s+"," ",str(s).replace("\u00a0"," ")).strip() if s is not None else ""

def pfloat(s):
    if s is None or clean(s)=="" or clean(s) in {"-","–","—"}:
        return None
    raw = str(s)
    if "," in raw and "." in raw:
        raw = raw.replace(".", "").replace(",", ".")
    else:
        raw = raw.replace(",", "")
    raw = re.sub(r"[^\d\.\-]", "", raw)
    try:
        return float(raw)
    except:
        return None

def parse_date(s):
    s = clean(s)
    for dayfirst in (False, True):
        try:
            return dtparse.parse(s, dayfirst=dayfirst, fuzzy=True)
        except:
            pass
    return None

def xnpv(rate, cashflows):
    t0 = min(d for d,_ in cashflows)
    days = 365.2425
    return sum(amt/((1+rate)**((d-t0).days/days)) for d,amt in cashflows)

def xirr(cashflows):
    if not cashflows:
        return None
    lo, hi = -0.9999, 10.0
    f_lo, f_hi = xnpv(lo, cashflows), xnpv(hi, cashflows)
    tries = 0
    while f_lo*f_hi > 0 and tries < 25:
        hi *= 1.5
        f_hi = xnpv(hi, cashflows); tries += 1
        if not math.isfinite(f_hi):
            break
    if f_lo*f_hi > 0:
        return None
    for _ in range(120):
        mid = (lo+hi)/2
        f   = xnpv(mid, cashflows)
        if abs(f) < 1e-6:
            return mid
        if f*f_lo > 0:
            lo, f_lo = mid, f
        else:
            hi, f_hi = mid, f
    return (lo+hi)/2

def ks_pme(cashflows, index_df):
    """
    Kaplan-Schoar PME: sum(CF_t * I_T/I_t) / sum(-Contr_t * I_T/I_t)
    index_df precisa ter colunas: date (datetime) e index (nível acumulado)
    """
    if index_df is None or index_df.empty:
        return None
    idx = index_df.sort_values("date")
    T   = idx["index"].iloc[-1]

    def level_on(d):
        i = idx[idx["date"] <= d]
        if i.empty:
            i = idx.iloc[:1]  # fallback: usa primeiro nível
        return i["index"].iloc[-1]

    num = 0.0; den = 0.0
    for d,amt in cashflows:
        L = level_on(d)
        adj = amt * (T / L)
        if amt >= 0:
            num += adj
        else:
            den += -adj
    if den == 0:
        return None
    return num / den

# --------------- CDI (série) ---------------
def fetch_cdi_index():
    """
    Usa API (MaisRetorno) para pegar CDI.
    Se 'c' vier como taxa/variação diária, acumula para formar um índice base 100.
    """
    url = "https://api.maisretorno.com/v3/indexes/quotes/cdi"
    r = requests.get(url, timeout=30, headers={"Accept": "application/json"})
    r.raise_for_status()
    dados = r.json()

    if isinstance(dados, dict) and "quotes" in dados:
        df_q = pd.DataFrame(dados["quotes"])
    else:
        df_q = pd.DataFrame(pd.DataFrame(dados)["quotes"].tolist())

    # campos esperados: 'd' (ms epoch), 'c' (valor)
    df_q["date"]  = pd.to_datetime(df_q["d"], unit="ms", errors="coerce")
    df_q["value"] = pd.to_numeric(df_q["c"], errors="coerce")
    df_q = df_q.dropna(subset=["date","value"]).sort_values("date")

    # Se parecer variação pequena, acumula; caso contrário, assume que já é nível
    if df_q["value"].median() < 5:
        base = 100.0
        levels = [base]
        for v in df_q["value"].tolist():
            levels.append(levels[-1] * (1 + float(v)/100.0))
        df_q["index"] = levels[1:]
    else:
        df_q["index"] = df_q["value"]

    return df_q[["date","index"]]

# --------------- Scraper Playwright ---------------
async def scrape_vehicle_financials():
    """
    Abre o FUND_URL, entra no Vehicle e extrai a seção Financials.
    Tenta DOM; se falhar, usa fallback por texto + regex.
    Retorna DataFrame com colunas: date, spec, contr, distr, contr_adj, distr_adj, nav
    """
    async with async_playwright() as p:
        ctx = await p.chromium.launch_persistent_context(USER_DATA, headless=HEADLESS)
        try:
            page = ctx.pages[0] if ctx.pages else await ctx.new_page()
            page.set_default_timeout(120_000)

            await page.goto(FUND_URL, wait_until="domcontentloaded", timeout=120_000)
            try:
                await page.wait_for_load_state("load", timeout=60_000)
            except:
                pass

            # tenta clicar no card/linha do vehicle
            try:
                await page.get_by_text("ANGRA INFRA", exact=False).first.click()
            except:
                pass  # pode já estar na página

            # garante que carregou algo com cara de Financials
            try:
                await page.wait_for_function(
                    "document.body && /Vehicle\\s*Name:|Financials|Date\\s+Specification\\s+Contribution\\(f\\)/i.test(document.body.innerText)",
                    timeout=90_000
                )
            except:
                raise RuntimeError("Não consegui abrir a página do Vehicle/Financials. Verifique login.")

            # rola até Financials
            try:
                loc = page.get_by_text("Financials", exact=False).first
                await loc.scroll_into_view_if_needed()
                await page.wait_for_timeout(1200)
            except:
                for _ in range(6):
                    await page.mouse.wheel(0, 2000)
                    await page.wait_for_timeout(500)

            # 1) tenta extrair via DOM
            rows = []
            try:
                container = page.locator("xpath=//*[contains(., 'Date') and contains(., 'Net Asset Value')]").last
                table = container.locator("xpath=.//table[.//th[contains(.,'Date')]]").first
                if await table.count() == 0:
                    grid = container.locator("xpath=.//*[contains(., 'Date')]/following::*").first
                    trs = grid.locator("tr")
                    if await trs.count() > 0:
                        for i in range(await trs.count()):
                            trow = trs.nth(i)
                            cells = trow.locator("xpath=.//th|.//td")
                            cols = [clean(await cells.nth(j).inner_text()) for j in range(await cells.count())]
                            rows.append(cols)
                    else:
                        divrows = grid.locator("xpath=.//*[self::div or self::li][descendant::*[contains(text(),'Jan') or contains(text(),'Feb') or contains(text(),'Mar') or contains(text(),'Dec') or contains(text(),', 20')]]")
                        n = await divrows.count()
                        for i in range(n):
                            txt = clean(await divrows.nth(i).inner_text())
                            rows.append(re.split(r"\s{2,}", txt))
                else:
                    tr = table.locator("xpath=.//tr")
                    n = await tr.count()
                    for i in range(n):
                        trow = tr.nth(i)
                        cells = trow.locator("xpath=.//th|.//td")
                        cols = [clean(await cells.nth(j).inner_text()) for j in range(await cells.count())]
                        rows.append(cols)
            except:
                rows = []

            def shape_dom_rows(rows):
                if not rows:
                    return None
                header_idx = None
                for i, r in enumerate(rows):
                    line = " ".join(r).lower()
                    if "date" in line and "contribution" in line and "net asset value" in line:
                        header_idx = i; break
                if header_idx is None:
                    return None
                data = rows[header_idx+1:]
                out = []
                for r in data:
                    if len(r) < 7:
                        continue
                    d = parse_date(r[0])
                    if not d:
                        continue
                    out.append({
                        "date": d,
                        "spec": clean(r[1]),
                        "contr": pfloat(r[2]),
                        "distr": pfloat(r[3]),
                        "contr_adj": pfloat(r[4]),
                        "distr_adj": pfloat(r[5]),
                        "nav": pfloat(r[6]),
                    })
                return pd.DataFrame(out) if out else None

            df_dom = shape_dom_rows(rows)

            # 2) fallback texto+regex
            if df_dom is None or df_dom.empty:
                block_text = await page.evaluate("""
                    () => {
                      const nodes = Array.from(document.querySelectorAll('*'))
                        .filter(n => /Date\\s+Specification\\s+Contribution\\(f\\).*Net\\s+Asset\\s+Value/i.test(n.textContent || ''));
                      if (nodes.length) return nodes[0].innerText;
                      return document.body.innerText;
                    }
                """)
            else:
                block_text = None
        finally:
            await ctx.close()

    if df_dom is not None and not df_dom.empty:
        return df_dom

    # --- regex fallback ---
    header_rgx = re.compile(r"Date\s+Specification\s+Contribution\(f\)\s+Distribution\(f\)\s+Adjusted\s+Contr.*?Net\s+Asset\s+Value", re.I)
    m = header_rgx.search(block_text or "")
    if not m:
        raise RuntimeError("Não encontrei o cabeçalho da tabela de Financials (nem por DOM nem por texto).")
    block = (block_text or "")[m.end():]
    cut = re.search(r"(?:\n|\r)(?:Sub\s+Total|Total)\b", block, flags=re.I)
    if cut:
        block = block[:cut.start()]

    line_rgx = re.compile(
        r"(\d{1,2}\s+\w{3},\s+\d{4})\s+([A-Za-zÀ-ÿ ]+?)\s+([\-0-9\.,]+)\s+([\-0-9\.,]+)\s+([\-0-9\.,]+)\s+([\-0-9\.,]+)\s+([\-0-9\.,]+)",
        re.I
    )
    out = []
    for mm in line_rgx.finditer(block):
        d = parse_date(mm.group(1))
        if not d:
            continue
        out.append({
            "date": d,
            "spec": clean(mm.group(2)),
            "contr": pfloat(mm.group(3)),
            "distr": pfloat(mm.group(4)),
            "contr_adj": pfloat(mm.group(5)),
            "distr_adj": pfloat(mm.group(6)),
            "nav": pfloat(mm.group(7)),
        })
    df = pd.DataFrame(out).dropna(subset=["date"]).sort_values("date").reset_index(drop=True)
    if df.empty:
        raise RuntimeError("Achei o bloco de Financials, mas não consegui ler nenhuma linha.")
    return df

# --------------- cálculo das métricas + séries ---------------
def compute_metrics_and_series(fin: pd.DataFrame, cdi_idx: pd.DataFrame | None):
    fin = fin.sort_values("date").reset_index(drop=True)

    # métricas agregadas (as-of último registro)
    paid_in       = fin["contr"].fillna(0).sum()
    distributions = fin["distr"].fillna(0).sum()
    nav_last      = fin["nav"].dropna().iloc[-1] if fin["nav"].notna().any() else 0.0

    paid_in_a       = fin["contr_adj"].fillna(0).sum()
    distributions_a = fin["distr_adj"].fillna(0).sum()

    def div(a,b): return (a/b) if b not in (None,0) else None

    agg = {
        "asof_date": fin["date"].iloc[-1],
        "paid_in": paid_in,
        "distributions": distributions,
        "NAV_last": nav_last,
        "TVPI": div(distributions + nav_last, paid_in),
        "DPI":  div(distributions, paid_in),
        "RVPI": div(nav_last, paid_in),
        "paid_in_adj": paid_in_a,
        "distributions_adj": distributions_a,
        "TVPI_adj": div(distributions_a + nav_last, paid_in_a),
        "DPI_adj":  div(distributions_a, paid_in_a),
        "RVPI_adj": div(nav_last, paid_in_a),
    }

    # IRR (as-of)
    cf = [(r.date, -(r.contr or 0.0)) for r in fin.itertuples()] + \
         [(r.date,  (r.distr or 0.0)) for r in fin.itertuples()]
    if nav_last:
        cf.append((fin["date"].iloc[-1], nav_last))
    agg["IRR"] = xirr(cf)

    cf_a = [(r.date, -(r.contr_adj or 0.0)) for r in fin.itertuples()] + \
           [(r.date,  (r.distr_adj or 0.0)) for r in fin.itertuples()]
    if nav_last:
        cf_a.append((fin["date"].iloc[-1], nav_last))
    agg["IRR_adj"] = xirr(cf_a)

    # Séries anuais (DPI/RVPI/TVPI por ano) – cumulativos até 31/12
    years = sorted(set(d.year for d in fin["date"]))
    rows = []
    for y in years:
        cut = fin[fin["date"] <= dt.datetime(y,12,31)]
        if cut.empty:
            continue
        pi   = cut["contr"].fillna(0).sum()
        dist = cut["distr"].fillna(0).sum()
        navy = cut["nav"].dropna().iloc[-1] if cut["nav"].notna().any() else None
        rows.append({
            "year": y,
            "paid_in": pi,
            "distributions": dist,
            "NAV": navy,
            "DPI": div(dist, pi),
            "RVPI": div(navy, pi) if navy is not None else None,
            "TVPI": div((dist + (navy or 0)), pi),
        })
    ratios_by_year = pd.DataFrame(rows)

    # IRR “as-of por ano”
    irr_rows = []
    for y in years:
        cut = fin[fin["date"] <= dt.datetime(y,12,31)]
        if cut.empty:
            continue
        navy = cut["nav"].dropna().iloc[-1] if cut["nav"].notna().any() else 0.0
        cf_y = [(r.date, -(r.contr or 0.0)) for r in cut.itertuples()] + \
               [(r.date,  (r.distr or 0.0)) for r in cut.itertuples()]
        if navy:
            cf_y.append((dt.datetime(y,12,31), navy))
        irr_rows.append({"year": y, "IRR_asof": xirr(cf_y)})
    irr_by_year = pd.DataFrame(irr_rows)

    # PME (CDI)
    pme_cdi_asof = None
    pme_cdi_by_year = pd.DataFrame()
    if cdi_idx is not None and not cdi_idx.empty:
        pme_cdi_asof = ks_pme(cf, cdi_idx)
        out = []
        for y in years:
            cut = fin[fin["date"] <= dt.datetime(y,12,31)]
            if cut.empty:
                continue
            navy = cut["nav"].dropna().iloc[-1] if cut["nav"].notna().any() else 0.0
            cf_y = [(r.date, -(r.contr or 0.0)) for r in cut.itertuples()] + \
                   [(r.date,  (r.distr or 0.0)) for r in cut.itertuples()]
            if navy:
                cf_y.append((dt.datetime(y,12,31), navy))
            out.append({"year": y, "PME_CDI": ks_pme(cf_y, cdi_idx)})
        pme_cdi_by_year = pd.DataFrame(out)

    agg["PME_CDI"] = pme_cdi_asof
    return pd.DataFrame([agg]), ratios_by_year, irr_by_year, pme_cdi_by_year

# --------------- runner ---------------
async def main():
    fin = await scrape_vehicle_financials()
    cdi_idx = fetch_cdi_index()

    metrics_df, ratios_year_df, irr_year_df, pme_cdi_year_df = compute_metrics_and_series(fin, cdi_idx)

    with pd.ExcelWriter(OUTPUT_XLSX, engine="openpyxl", mode="w") as xw:
        fin.to_excel(xw, sheet_name="financials", index=False)
        metrics_df.to_excel(xw, sheet_name="metrics_asof", index=False)
        ratios_year_df.to_excel(xw, sheet_name="ratios_by_year", index=False)
        irr_year_df.to_excel(xw, sheet_name="irr_by_year", index=False)
        pme_cdi_year_df.to_excel(xw, sheet_name="pme_cdi_by_year", index=False)
        cdi_idx.to_excel(xw, sheet_name="cdi_index", index=False)

    print("✅ OK! Planilha salva em:", OUTPUT_XLSX)
    print("Linhas de Financials:", len(fin))

# No notebook:  await main()





In [24]:
# --- PATCH: 2FA manual + sessão persistente e storageState ---

import os, re, asyncio, datetime as dt
from pathlib import Path
from getpass import getpass
import pandas as pd
from playwright.async_api import async_playwright, TimeoutError as PWTimeout

# Parâmetros (já usa seu perfil persistente)
USER_DATA    = os.path.join(Path.home(), ".pw-angra")
FUND_URL     = "https://pebay.info/backoffice/fund/angra-infra"
OUTPUT_DIR   = r"C:\Users\otavi\OneDrive\Documentos\Estudos_2025\COINP\PROJECOES\BANCODEDADOS\FIPS"
HEADLESS     = False
STATE_JSON   = os.path.join(USER_DATA, "pebay_state.json")  # estado reutilizável

os.makedirs(OUTPUT_DIR, exist_ok=True)
os.makedirs(USER_DATA, exist_ok=True)

async def is_connected(page):
    try:
        return await page.evaluate(
            "() => !!(window.Meteor && Meteor.status && Meteor.status().connected)"
        )
    except:
        # fallback: testa presença de elementos do backoffice
        try:
            return await page.evaluate(
                "() => /backoffice/i.test(location.pathname)"
            )
        except:
            return False

async def wait_manual_2fa(page, ctx, max_seconds=240):
    """
    Mostra o browser para você fazer o 2FA; aguarda até autenticar ou estourar o tempo.
    Ao autenticar, salva storageState para reuso.
    """
    print("🔐 Aguarde/autentique o 2FA na janela aberta. Tenho até", max_seconds, "s.")
    deadline = dt.datetime.now() + dt.timedelta(seconds=max_seconds)

    # pequenos hints de seletores de 2FA, caso apareçam
    possible_2fa_selectors = [
        'input[name*="code"]', 'input[id*="code"]', 'input[type="tel"]',
        'text=Two-Factor', 'text=Verificação em duas etapas', 'text=Authenticator'
    ]

    while dt.datetime.now() < deadline:
        if await is_connected(page):
            try:
                await ctx.storage_state(path=STATE_JSON)
            except Exception as e:
                print("Aviso: não consegui salvar storageState:", e)
            print("✅ 2FA concluído. Sessão autenticada.")
            return True

        # Se aparecer campo de código, dá foco (só ajuda visualmente)
        try:
            for sel in possible_2fa_selectors:
                loc = page.locator(sel)
                if await loc.count():
                    try:
                        await loc.first.focus()
                        break
                    except:
                        pass
        except:
            pass

        await asyncio.sleep(1.5)

    raise RuntimeError("Tempo esgotado aguardando 2FA manual.")

async def open_context_and_login():
    """
    Se existir STATE_JSON, tenta usá-lo; senão, abre contexto persistente e espera 2FA.
    Retorna (ctx, page) autenticados.
    """
    p = await async_playwright().start()

    # Preferir contexto persistente para reaproveitar cookies e SSO
    ctx = await p.chromium.launch_persistent_context(
        USER_DATA,
        headless=HEADLESS,
        args=["--start-maximized"],
        no_viewport=True,
        # slow_mo=50,  # opcional: mais “humano” para 2FA
    )
    page = ctx.pages[0] if ctx.pages else await ctx.new_page()

    # Vai para o destino
    await page.goto(FUND_URL, wait_until="domcontentloaded", timeout=120_000)
    try:
        await page.wait_for_load_state("load", timeout=60_000)
    except:
        pass

    # Já está logado?
    if await is_connected(page):
        return p, ctx, page

    # Caso não esteja, aguarda o 2FA manual
    await wait_manual_2fa(page, ctx, max_seconds=240)

    # Garante que estamos na página do veículo após login
    if not re.search(r"/fund/angra-infra", page.url, re.I):
        await page.goto(FUND_URL, wait_until="domcontentloaded", timeout=120_000)
        try:
            await page.wait_for_load_state("load", timeout=60_000)
        except:
            pass

    return p, ctx, page


In [25]:
await main()

RuntimeError: Não consegui abrir a página do Vehicle/Financials. Verifique login.